In [ ]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import pydash

load_dotenv()

client = genai.Client()

subtraction_fn = {
  "name": "subtract",
  "description": "Subtract two numbers",
  "parameters": {
    "type": "object",
    "properties": {
      "a": {
        "type": "number",
        "description": "The first number"
      },
      "b": {
        "type": "number",
        "description": "The second number"
      }
    },
    "required": ["a", "b"]
  }
}

def subtract(a: float, b: float) -> float:
    return a - b


subtraction_declaration = types.FunctionDeclaration(**subtraction_fn)
tools = types.Tool(function_declarations=[subtraction_declaration])
config = types.GenerateContentConfig(tools=[tools])

messages_record = [
    types.Content(
        role="user",
        parts=[types.Part(text="What is 9.11 - 9.9?")]
    )
]

while True:
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=messages_record, # type: ignore
        config=config,
    )
    print(response)

    function_call = pydash.get(response, "candidates[0].content.parts[0].function_call")
    # 判断 function_call 并执行
    if function_call and function_call.name == "subtract":
        result = subtract(**function_call.args)
        print(f"Result: {result}")

        # 将函数调用加入上下文
        messages_record.append(pydash.get(response, "candidates[0].content"))

        # 将函数调用结果加入上下文
        messages_record.append(types.Content(
            role="user",
            parts=[types.Part.from_function_response(
                name="subtract",
                response={"result": result}
            )]
        ))

    else:
        print(f"Final answer: {response.text}")
        break
